In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import *
import csv


Read merged data file with charging stations and ACS data

In [43]:
#import the merged charging station and ACS data
lines = open('data/add_0_stations.csv').read().splitlines()
#Remove the first row, it's just the headings
lines.pop(0)

zip_code, count, population, male_pct, age, college_pct, income, home_pct = [], [], [], [], [], [], [], []
for line in lines:
    items = line.split(',')
    zip_code.append(items[0])
    count.append(float(items[1]))
    population.append(float(items[2]))
    male_pct.append(float(items[3]))
    age.append(items[4])
    college_pct.append(items[5])
    income.append(items[6])
    home_pct.append(float(items[7]))
    
# combine lists to numpy arrays for data and labels
Y=np.array(count)
# X=np.vstack((population,male_pct, age, college_pct, income, home_pct))
X=np.vstack((population, male_pct, home_pct))
X=np.transpose(X)
print Y.shape, X.shape, X[0], Y[0]

# add shuffle code before splitting the data into train and dev sets
# easiest shuffle utility is sklearn shuffle function
from sklearn.utils import shuffle

# shuffle two arrays in unison
X, Y = shuffle(X, Y, random_state=0)
print X.shape, Y.shape, X[0], Y[0]
# determine the data type of the two arrays
print X.dtype, Y.dtype
# convert to strings
# must handle missing values first by making missing values zeros
X[X==''] = '0'
# now convert to float or leave them alone - converting now to make the code cleaner
X1=X.astype(np.float)
print X1.dtype

# Split the imported data into training 75% and development data 25%

split = 3*len(zip_code)/4

print len(zip_code), split

train_data, train_labels = X[:split], Y[:split]
dev_data, dev_labels = X[split:], Y[split:]

print train_data.shape, train_labels.shape
print dev_data.shape, dev_labels.shape
print train_data[0], train_labels[0]
print dev_data[0], dev_labels[0]
# dev_dat=np.array(dev_data)
# print dev_dat.sum()

(32517,) (32517, 3) [  6.40200000e+03   4.97188379e-01   0.00000000e+00] 1.0
(32517, 3) (32517,) [  2.77250000e+04   4.84147881e-01   7.08960989e-01] 1.0
float64 float64
float64
32517 24387
(24387, 3) (24387,)
(8130, 3) (8130,)
[ 0.  0.  0.] 1.0
[  1.64680000e+04   5.06922516e-01   6.86856451e-01] 0.0


In [40]:
# try lineary regression mode
from sklearn import datasets, linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_data, train_labels)

# The coefficients
print('Coefficients: \n', regr.coef_)

# score
print('Variance score: %.2f' % regr.score(dev_data, dev_labels))


('Coefficients: \n', array([  2.53171301e-05,   1.84813537e-01,  -1.17812150e+00]))
Variance score: 0.15


In [45]:
# fit the first model with logistic regression 
#
logreg = LogisticRegression(penalty='l1', C=1.0, tol=0.01) 
logreg.fit(train_data, train_labels)
print logreg.score(dev_data, dev_labels)

0.863099630996
